In [1]:
# Instalar Java Tool Kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar Spark
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompactar Spark
!tar xf /content/spark-3.4.1-bin-hadoop3.tgz

# Instalar lib findspark que ajuda a localizar o Spark no sistema e importa-lo como uma biblioteca regular
!pip install -q findspark

In [2]:
# Lib os - Permite Interagir com S.O - Cria variaveis de ambiente necessarias
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [3]:
# Inicializa Findspark - Facilita Uso de Spark em Notebook
import findspark
findspark.init()

In [4]:
# Start SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName('Rate Source')\
        .master('local')\
        .getOrCreate()


In [5]:
# Cria Dataframe a partir de Dados Gerados Automaticamente pela Fonte Rate
# Define Geração de Dados a Cada 2 Segundos
df = spark.readStream.\
           format('rate').\
           option('rowsPerSecond', 2). \
           load()

In [6]:
# Confirma Objeto ser Streaming e Apresenta Descrição do Tipo de Objeto
print(df.isStreaming,'para Streaming - Tipo:',type(df))

True para Streaming - Tipo: <class 'pyspark.sql.dataframe.DataFrame'>


In [7]:
# Print Schema
df.printSchema

<bound method DataFrame.printSchema of DataFrame[timestamp: timestamp, value: bigint]>

In [8]:
# Saida de Dados - Modo Append -
query = df \
        .writeStream \
        .format('memory') \
        .outputMode('append') \
        .queryName('tb_dados') \
        .start()

In [10]:
# Consulta Fluxo Continuo de Dados
spark.sql("select * from tb_dados").show(truncate=False)

+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2023-11-06 04:50:17.821|0    |
|2023-11-06 04:50:18.321|1    |
|2023-11-06 04:50:18.821|2    |
|2023-11-06 04:50:19.321|3    |
|2023-11-06 04:50:19.821|4    |
|2023-11-06 04:50:20.321|5    |
|2023-11-06 04:50:20.821|6    |
|2023-11-06 04:50:21.321|7    |
|2023-11-06 04:50:21.821|8    |
|2023-11-06 04:50:22.321|9    |
|2023-11-06 04:50:22.821|10   |
|2023-11-06 04:50:23.321|11   |
|2023-11-06 04:50:23.821|12   |
|2023-11-06 04:50:24.321|13   |
+-----------------------+-----+



In [13]:
# Consulta Qtde de Linhas
spark.sql("select count(*) from tb_dados").show()

+--------+
|count(1)|
+--------+
|      42|
+--------+



In [14]:
# Parar SparkContext - Somente Executar se Precisar Reprocessar o Streaming
spark.stop()